In [1]:
%config Completer.use_jedi = False
from importlib import reload

In [2]:
# import statsmodels.api as sm

import pandas as pd
import numpy as np

# import matplotlib
import matplotlib.pyplot as plt

# from IPython.display import Image, display

In [3]:
pd.set_option("mode.chained_assignment", None)

In [4]:
import util.load as load_util
import util.explore as explore_util
import util.experiment as experiment

In [5]:
from econml.dml import LinearDML, SparseLinearDML, NonParamDML
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

### Load in data frames

1. Load in panel assembled by DG, and country code cross-matches
2. For each country-year, calculate mean growth in education indicators at year + lag

In [6]:
project_df = load_util.load_projects() # loads in aid data projects
edu_treatment_df = load_util.assemble_sector_ratings(project_df, 'Education').fillna(0) # loads in 
edu_treatment_df.head()

,country_code,end_year,num_projs,total_proj_size,w_avg_rating,min_rating,max_rating
0,AAB,2003.0,1,1762955.2,6.0,6.0,6.0
1,AFG,1984.0,1,0.0,0.0,0.0,0.0
2,AFG,2004.0,1,2560000.0,0.0,0.0,0.0
3,AFG,2005.0,1,4413731.0,4.5,4.5,4.5
4,AFG,2007.0,1,3134497.1,4.5,4.5,4.5


In [7]:
# need: reassemble oda per capita from aid_data file.

In [35]:
edu_treatment_df.w_avg_rating.describe()

count    723.000000
mean       4.194083
std        1.269325
min        0.000000
25%        3.848989
50%        4.800000
75%        5.000000
max        6.000000
Name: w_avg_rating, dtype: float64

In [ ]:
recreate_df = False

if recreate_df:
    panel_df, panel_source = experiment.assemble_replication_panel('education')
    edu_df = panel_df.merge(edu_treatment_df, how='left', left_on=['year', 'country'], right_on=['end_year', 'country_code'])
    treatment_cols = [col for col in edu_treatment_df.columns if col not in ["end_year", "country_code"]]
    edu_df[treatment_cols] = edu_df[treatment_cols].fillna(0)
else:
    edu_df = pd.read_csv('../data/transformed_data/education_df.csv', index_col=0)
    # deal with some legacy (quicker than regenerating)
    edu_df = edu_df.drop(columns=[f"education_lag_{i}_growth" for i in range(1, 10)], errors="ignore")
    edu_df = edu_df.drop(columns=[f"education_lag_{i}_count" for i in range(1, 10)], errors="ignore")
    edu_df = edu_df.drop(columns=["education_lag_-4_count", "mean_pc_last_5", "future_edu_ner", "lagged_edu_ner", "ner_growth"], errors="ignore")

### Construct lagged indicators, fille in needed columns

In [ ]:
edu_df.project_completed_year.value_counts()

Generate some necessary feature and outcome columns

In [ ]:
edu_df = experiment.add_project_and_aid_cols(edu_df)

if 'edu_ner_lag5' not in edu_df:
    print('Generating past net enrollment rates')
    edu_df = explore_util.lag_variable_simple(df, 'edu_ner', 5)

if 'future_edu_ner' not in edu_df:
    print('Generating future net enrollment rates')
    edu_df = explore_util.lag_variable_simple(df, 'edu_ner', -5)
    edu_df = edu_df.rename(columns = { 'edu_ner_lag-5': 'future_edu_ner'})

In [ ]:
edu_df[df.edu_ner_lag5.notna()][['year', 'country', 'edu_ner', 'edu_ner_lag5', 'pc_commit_education', 'education_mean_pc_rolling_5', 'education_mean_pc_rolling_5_lag1']].head()

In [ ]:
store_df = False

if store_df:
    edu_df.to_csv('../data/transformed_data/education_df.csv')

## Education: replicate original, then probe the specification

Education: Specification 2
```
regress  last_ner <- first_ner pc_commit_education [per capita commitment amount=
        edu_share_gov_exp edu_pupil_teacher young_population
        gdp_pc_ppp cash_surplus_deficit inflation trade_share_gdp
        freedom_house i.period i.ncountrycode if countrytoinclude == 1, r
```

In [ ]:
edu_df['period'] = round((edu_df.year - 1900) / 5) - 10
edu_df['prior_ner_growth'] = edu_df['edu_ner'] / edu_df['edu_ner_lag5']
edu_df = explore_util.lag_variable_simple(edu_df, 'pc_commit_education', 5)

In [ ]:
edu_df[(edu_df.education_mean_pc_rolling_5_lag1.notna()) & (edu_df.country == 'CHN')][['country', 'year', 'edu_ner', 'mean_pc_last_5', 'pc_commit_education', 'education_mean_pc_rolling_5_lag1']]

In [ ]:
data_cols = ['edu_ner', 'mean_pc_last_5',
             'edu_share_gov_exp', 'edu_pupil_teacher', 'young_population', 'gdp_pc_ppp', 
             'cash_surplus_deficit', 'inflation', 'trade_share_gdp',
             'freedom_house', 'prior_ner_growth', 'prior_4year_growth']
    
edu_treatment_col = 'mean_pc_last_5'

initial_drop = ['prior_ner_growth', 'edu_share_gov_exp', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth']
r_est = experiment.evaluate_treatment(edu_df, 'edu_ner', edu_treatment_col, data_cols,
                          remove_feature_cols=initial_drop,
                          add_country_feffects=True, add_constant=False, log_target=True, log_treatment=True,
                                     add_period_feffects=False)

experiment.extract_treatment_results('Replication', r_est, 'edu_ner', edu_treatment_col, data_cols, None)

In [ ]:
print(r_est.summary())

In [ ]:
search_grid = {
    'Straight replication': {
        'target_col': 'edu_ner', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=False, log_target=True, log_treatment=True,
            remove_feature_cols=['prior_ner_growth', 'edu_share_gov_exp', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth']
        )
    },
    'Include rating information': {
        'target_col': 'edu_ner', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=False, log_target=True, log_treatment=True,
            remove_feature_cols=['prior_ner_growth', 'edu_share_gov_exp', 'prior_4year_growth']
        )
    },
    'Include period fixed effects': {
        'target_col': 'edu_ner', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=True, log_target=True, log_treatment=True, add_period_feffects=True,
            remove_feature_cols=['prior_ner_growth', 'edu_share_gov_exp', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth']
        )        
    },
    'Include prior growth across education outcomes': {
        'target_col': 'edu_ner', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=False, log_target=True, log_treatment=True,
            remove_feature_cols=['prior_ner_growth', 'edu_share_gov_exp', 'satisfactory_proj', 'w_avg_rating']
        )
    },
    'Properly include govt share spend': {
        'target_col': 'edu_ner', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=False, log_target=True, log_treatment=True,
            remove_feature_cols=['prior_ner_growth', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth']
        )
    },
    'Use growth in NER as target': {
        'target_col': 'prior_ner_growth', 
        'treatment_col': edu_treatment_col,
        'est_kw_args': dict(
            add_country_feffects=True, add_constant=True, log_target=True, log_treatment=True,
            remove_feature_cols=['edu_ner', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth']
        )

    }
}

In [ ]:
treatment_search_result = []
estimators = {}
for label, args in search_grid.items():
    est = experiment.evaluate_treatment(edu_df, args['target_col'], args['treatment_col'], data_cols, **args['est_kw_args'])
    results = experiment.extract_treatment_results(label, est, args['target_col'], args['treatment_col'], data_cols, args['est_kw_args'])
    treatment_search_result.append(results)
    estimators[label] = est

gsearch_results = pd.DataFrame(treatment_search_result)
gsearch_results

In [ ]:
write_results = False

if write_results:
    with open("./growth_not_abs_ner_target.txt", "w") as file:
        file.write(estimators['Use growth in NER as target'].summary().as_text())

    with open("./base_replication_full.txt", "w") as file:
        file.write(estimators['Straight replication'].summary().as_text())

    gsearch_results.to_csv('../data/results/education_model_crawl.csv', float_format='%.4f')

In [ ]:
display_repl_summary = True

if display_repl_summary:
    print("*** Standard Replication: ")
    print(r_est.summary())
    
    print("*** Replication with average rating: ")
    print(estimators['Include rating information'].summary())

### Some EconML work now

In [ ]:
feature_cols = ['edu_share_gov_exp', 'edu_pupil_teacher', 'young_population', 'gdp_pc_ppp', 
             'cash_surplus_deficit', 'inflation', 'trade_share_gdp', 'freedom_house']

In [ ]:
rating_col = 'w_avg_rating'
treatment_col = 'mean_pc_last_5'
target_col = 'edu_ner'

# feature_cols = [col for col in data_cols if col != treatment_col and col != target_col]
Y, T, X, W = experiment.assemble_econml_tuples(edu_df, target_col='edu_ner', treatment_col=treatment_col, 
                                               feature_cols=feature_cols + [rating_col])

est = LinearDML(model_y=LassoCV(), model_t=LassoCV())
est.fit(Y, T, X=X, W=W)
print(est.summary())

In [ ]:
rating_col = 'w_avg_rating'
magnitude_col = 'mean_pc_last_5'
target_col = 'edu_ner'

# feature_cols = [col for col in data_cols if col not in [rating_col, magnitude_col, target_col]]
Y, T, X, W = experiment.assemble_econml_tuples(edu_df, target_col=target_col, treatment_col=rating_col, 
                                               feature_cols=feature_cols + [magnitude_col])

est = LinearDML(model_y=LassoCV(), model_t=LassoCV())
est.fit(Y, T, X=X, W=W)
print(est.summary())

## Health

Process:

1. Repeat outcome variable formation, using lagged construction
2. Construct sectoral aid per capita using utilities
3. Construct specification, using Diana's original notebook

In [36]:
health_treatment_df = load_util.assemble_sector_ratings(project_df, 'Health').fillna(0)
health_treatment_df.head()

,country_code,end_year,num_projs,total_proj_size,w_avg_rating,min_rating,max_rating
0,AAB,2003.0,1,1684568.08,6.0,6.0,6.0
1,AFG,2006.0,1,0.00,4.5,4.5,4.5
2,AFG,2009.0,1,0.00,6.0,6.0,6.0
3,AFG,2010.0,1,7722667.00,6.0,6.0,6.0
4,AFG,2011.0,1,0.00,6.0,6.0,6.0


In [37]:
health_treatment_df.w_avg_rating.describe()

count    844.000000
mean       4.331075
std        1.346807
min        0.000000
25%        3.878389
50%        4.500000
75%        5.000000
max        6.000000
Name: w_avg_rating, dtype: float64

In [45]:
hp_df, hp_source = experiment.assemble_replication_panel('health', reload=False)

Assembling main panel
Loaded main panel, constructing next one


In [51]:
health_df = hp_df.merge(health_treatment_df, how='left', left_on=['year', 'country'], right_on=['end_year', 'country_code'])

In [52]:
health_df.w_avg_rating.describe()

count    326.000000
mean       4.227413
std        1.383231
min        0.000000
25%        3.750000
50%        4.500000
75%        5.000000
max        6.000000
Name: w_avg_rating, dtype: float64

In [38]:
recreate_health_df = False

if recreate_health_df:
    hp_df, hp_source = experiment.assemble_replication_panel('health')
    health_df = hp_df.merge(health_treatment_df, how='left', left_on=['year', 'country'], right_on=['end_year', 'country_code'])
    treatment_cols = [col for col in health_treatment_df.columns if col not in ["end_year", "country_code"]]
    health_df[treatment_cols] = edu_df[treatment_cols].fillna(0)
else:
    health_df = pd.read_csv('../data/transformed_data/health_df.csv', index_col=0)

In [53]:
health_df = experiment.add_project_and_aid_cols(health_df, sector='health')

treatment_cols = [col for col in health_treatment_df.columns if col not in ["end_year", "country_code"]]
health_df[treatment_cols] = health_df[treatment_cols].fillna(0)
print("Treatment counts: ", health_df.project_completed_year.value_counts())

Generating mean per capita commitments over prior years
Marking whether a satisfactory project concluded in that year
Taking maximum of weighted rating of concluded projects in prior period
Treatment counts:  False    8717
True      747
Name: project_completed_year, dtype: int64


In [54]:
health_df[health_df.w_avg_rating > 0].w_avg_rating.describe()

count    314.000000
mean       4.388970
std        1.129312
min        1.000000
25%        4.000000
50%        4.500000
75%        5.000000
max        6.000000
Name: w_avg_rating, dtype: float64

In [55]:
store_health_df = False

if store_health_df:
    health_df.to_csv('../data/transformed_data/health_df.csv')

In [56]:
# take rolling five year averages of everything, following paper
# note: not doing this for macro variables as justification does not make sense for those (measurement/volatility)
measured_cols = ['mortality_under5', 'fertility', 'hiv_prevalence']

for m_col in measured_cols:
    health_df[f"{m_col}_pavg"] = explore_util.rolling_country_agg(health_df, m_col, 5, "mean")
    health_df = explore_util.lag_variable_simple(health_df, f"{m_col}_pavg", 1)
    
macro_cols = ["gdp_pc_ppp", "population"]
for m_col in macro_cols:
    health_df = explore_util.lag_variable_simple(health_df, m_col, 1)

In [57]:
health_df['mean_pc_last_5'] = (
    health_df.pc_commit_health_lag5.notna() * health_df.health_mean_pc_rolling_5_lag1
).replace({ 0: np.nan })

health_df = explore_util.lag_variable_simple(health_df, "mortality_under5_pavg", 5)
health_df["lag_log_mort"] = np.log(health_df["mortality_under5_pavg_lag5"])
health_df['prior_mort_decline'] = health_df['mortality_under5_pavg'] / health_df['mortality_under5_pavg_lag5']

* Specification 1:

```qui regress mortality_under5 pc_commit_health lag_mortality_under5 ///
            lag_gdp_pc_ppp lag_fertility lag_population ///
            hiv_prevalence conflict i.period, r```

In [58]:
# health_df.columns

In [59]:
target_cols = ["mortality_under5_pavg"]
treatment_cols = ["mean_pc_last_5"]

momentum_cols = ["lag_log_mort"]
control_cols = ["hiv_prevalence_pavg_lag1", "fertility_pavg_lag1", "gdp_pc_ppp_lag1", "population_lag1", "conflict"]
rating_cols = ['health_satisfactory_proj', 'w_avg_rating']


health_data_cols = ["country"] + target_cols + treatment_cols + momentum_cols + control_cols + rating_cols

In [60]:
# first go for the paper
# replication_exclude = ['prior_mort_decline', 'w_avg_rating', 'satisfactory_proj', 'prior_4year_growth',
#                       'inflation', 'trade_share_gdp', 'freedom_house', 'mean_pc_last_5']

health_est = experiment.evaluate_treatment(health_df, 'mortality_under5_pavg', 'mean_pc_last_5', health_data_cols,
                            remove_feature_cols=["prior_mort_decline"], add_country_feffects=False, 
                            add_constant=False, log_target=True, log_treatment=True)

experiment.extract_treatment_results('Health Replication', health_est, 'mortality_under5_pavg', 'mean_pc_last_5', health_data_cols, None)

{'Label': 'Health Replication',
 'Regression P': 0.0,
 'Treatment significance': 4.905810680712437e-25,
 'Treatment coefficient': -0.025423821817695903,
 'Sig feature coefficient': {'lag_log_mort': 0.937,
  'hiv_prevalence_pavg_lag1': 0.0097,
  'fertility_pavg_lag1': 0.0356,
  'gdp_pc_ppp_lag1': -0.0,
  'population_lag1': -0.0},
 'Sig feature p-values': {'lag_log_mort': 0.0,
  'hiv_prevalence_pavg_lag1': 0.0,
  'fertility_pavg_lag1': 0.0,
  'gdp_pc_ppp_lag1': 0.0,
  'population_lag1': 0.0011},
 'Number significant FE': 0,
 'Mean coefficient on FE': 0,
 'Keyword args': None}

In [61]:
print(health_est.summary())

                                  OLS Regression Results                                  
Dep. Variable:     mortality_under5_pavg   R-squared (uncentered):                   0.999
Model:                               OLS   Adj. R-squared (uncentered):              0.999
Method:                    Least Squares   F-statistic:                          2.521e+05
Date:                   Tue, 24 Aug 2021   Prob (F-statistic):                        0.00
Time:                           14:38:47   Log-Likelihood:                          1282.4
No. Observations:                   1654   AIC:                                     -2547.
Df Residuals:                       1645   BIC:                                     -2498.
Df Model:                              9                                                  
Covariance Type:               nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.97

In [62]:
# now add in controls for macro conditions
cols_rep2 = health_data_cols + ['inflation', 'cash_surplus_deficit', 'trade_share_gdp']
health_est2 = experiment.evaluate_treatment(health_df, 'mortality_under5_pavg', 'mean_pc_last_5', cols_rep2,
                          add_country_feffects=False, add_constant=False, log_target=True, log_treatment=True)

experiment.extract_treatment_results('Health Replication with Macro', health_est2, 'mortality_under5_pavg', 'mean_pc_last_5', cols_rep2, None)

{'Label': 'Health Replication with Macro',
 'Regression P': 0.0,
 'Treatment significance': 4.9238152669813355e-21,
 'Treatment coefficient': -0.0317597980508285,
 'Sig feature coefficient': {'lag_log_mort': 0.9391,
  'hiv_prevalence_pavg_lag1': 0.0097,
  'fertility_pavg_lag1': 0.0373,
  'gdp_pc_ppp_lag1': -0.0,
  'population_lag1': -0.0,
  'inflation': 0.0002,
  'cash_surplus_deficit': -0.0016,
  'trade_share_gdp': -0.0003},
 'Sig feature p-values': {'lag_log_mort': 0.0,
  'hiv_prevalence_pavg_lag1': 0.0,
  'fertility_pavg_lag1': 0.0,
  'gdp_pc_ppp_lag1': 0.0,
  'population_lag1': 0.0011,
  'inflation': 0.0048,
  'cash_surplus_deficit': 0.0416,
  'trade_share_gdp': 0.0164},
 'Number significant FE': 0,
 'Mean coefficient on FE': 0,
 'Keyword args': None}

* Specification 6

```qui regress mortality_under5 pc_commit_health lag_mortality_under5 ///
            lag_gdp_pc_ppp lag_fertility lag_population ///
            lag_physicians_rate  lag_female_adult_literacy ///
            lag_access_water lag_access_sanitation ///
            hiv_prevalence conflict i.period i.nregionname, r```

In [63]:
'female_adult_literacy' in health_df

True

In [64]:
# 'female_adult_literacy',
cols_rep3 = health_data_cols + ['access_water', 'access_sanitation', 'physicians_rate']

health_est3 = experiment.evaluate_treatment(health_df, 'mortality_under5_pavg', 'mean_pc_last_5', 
                          cols_rep3, add_country_feffects=True, add_period_feffects=True,
                            add_constant=False, log_target=True, log_treatment=True)

experiment.extract_treatment_results('Health Replication 6', health_est3, 'mortality_under5_pavg', 'mean_pc_last_5', health_data_cols, None)

{'Label': 'Health Replication 6',
 'Regression P': 0.0,
 'Treatment significance': 2.1541154943776364e-20,
 'Treatment coefficient': -0.023413006226278434,
 'Sig feature coefficient': {'lag_log_mort': 0.7046,
  'hiv_prevalence_pavg_lag1': 0.0134,
  'fertility_pavg_lag1': 0.0896,
  'gdp_pc_ppp_lag1': -0.0,
  'population_lag1': 0.0},
 'Sig feature p-values': {'lag_log_mort': 0.0,
  'hiv_prevalence_pavg_lag1': 0.0,
  'fertility_pavg_lag1': 0.0,
  'gdp_pc_ppp_lag1': 0.0,
  'population_lag1': 0.0223},
 'Number significant FE': 100,
 'Mean coefficient on FE': 0.8408437654732221,
 'Keyword args': None}

In [65]:
print(health_est3.summary())

                              OLS Regression Results                             
Dep. Variable:     mortality_under5_pavg   R-squared:                       0.994
Model:                               OLS   Adj. R-squared:                  0.994
Method:                    Least Squares   F-statistic:                     2214.
Date:                   Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                           14:38:54   Log-Likelihood:                 2077.8
No. Observations:                   1617   AIC:                            -3930.
Df Residuals:                       1504   BIC:                            -3321.
Df Model:                            112                                         
Covariance Type:               nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [68]:
# now just with positive rating
cols_rep4 = health_data_cols + ['access_water', 'access_sanitation', 'physicians_rate']

health_est4 = experiment.evaluate_treatment(health_df[health_df.w_avg_rating > 0], 'mortality_under5_pavg', 'mean_pc_last_5', 
                          cols_rep3, add_country_feffects=False, add_period_feffects=False,
                            add_constant=False, log_target=True, log_treatment=True)

experiment.extract_treatment_results('Health Replication Only Rating', health_est3, 'mortality_under5_pavg', 'mean_pc_last_5', health_data_cols, None)

{'Label': 'Health Replication Only Rating',
 'Regression P': 0.0,
 'Treatment significance': 2.1541154943776364e-20,
 'Treatment coefficient': -0.023413006226278434,
 'Sig feature coefficient': {'lag_log_mort': 0.7046,
  'hiv_prevalence_pavg_lag1': 0.0134,
  'fertility_pavg_lag1': 0.0896,
  'gdp_pc_ppp_lag1': -0.0,
  'population_lag1': 0.0},
 'Sig feature p-values': {'lag_log_mort': 0.0,
  'hiv_prevalence_pavg_lag1': 0.0,
  'fertility_pavg_lag1': 0.0,
  'gdp_pc_ppp_lag1': 0.0,
  'population_lag1': 0.0223},
 'Number significant FE': 100,
 'Mean coefficient on FE': 0.8408437654732221,
 'Keyword args': None}

In [69]:
print(health_est4.summary())

                                  OLS Regression Results                                  
Dep. Variable:     mortality_under5_pavg   R-squared (uncentered):                   0.999
Model:                               OLS   Adj. R-squared (uncentered):              0.999
Method:                    Least Squares   F-statistic:                          3.029e+04
Date:                   Tue, 24 Aug 2021   Prob (F-statistic):                   2.79e-310
Time:                           14:40:17   Log-Likelihood:                          193.43
No. Observations:                    206   AIC:                                     -362.9
Df Residuals:                        194   BIC:                                     -322.9
Df Model:                             12                                                  
Covariance Type:               nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025      0.97

### Now trying EconML (on health)

In [71]:
health_est_ldml = LinearDML(model_y=LassoCV(), model_t=LassoCV())

In [72]:
health_df["log_mort"] = np.log(health_df["mortality_under5_pavg"])
health_df["log_mean_pc_last_5"] = np.log(health_df["mean_pc_last_5"])
health_df["log_proj_rating"] = np.log(health_df["w_avg_rating"].replace(0, np.nan)).fillna(0).fillna(0)

In [73]:
if "access_water" not in control_cols:
    control_cols += ["physicians_rate", "health_share_gov_exp"]

In [74]:
Y, T, X, W = experiment.assemble_econml_tuples(health_df, "log_mort", "log_mean_pc_last_5",
                                              control_cols + ["log_proj_rating"])
health_est_ldml.fit(Y, T, X=X, W=W)
health_est_ldml.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
hiv_prevalence_pavg_lag1,0.003,0.003,0.763,0.446,-0.003,0.008
fertility_pavg_lag1,0.13,0.01,12.896,0.0,0.114,0.147
gdp_pc_ppp_lag1,0.0,0.0,12.765,0.0,0.0,0.0
population_lag1,0.0,0.0,2.399,0.016,0.0,0.0
conflict,-0.142,0.0,-1029.564,0.0,-0.142,-0.142
physicians_rate,0.122,0.001,142.684,0.0,0.121,0.124
health_share_gov_exp,0.016,0.003,5.239,0.0,0.011,0.021
log_proj_rating,-0.015,0.001,-26.81,0.0,-0.016,-0.014
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-0.717,0.001,-487.149,0.0,-0.719,-0.715


In [75]:
health_df.w_avg_rating.describe()

count    9464.000000
mean        0.145619
std         0.812503
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: w_avg_rating, dtype: float64

In [76]:
health_df['w_avg_rating_std'] = (health_df['w_avg_rating'] - health_df['w_avg_rating'].mean()) / health_df['w_avg_rating'].std()
health_df['log_scale_avg_rating'] = experiment.safe_log(health_df, 'w_avg_rating_std')

invalid value encountered in log


In [77]:
Y, T, X, W = experiment.assemble_econml_tuples(health_df[health_df.w_avg_rating > 0], "log_mort", "log_scale_avg_rating",
                                              control_cols + ["log_mean_pc_last_5"])

health_est_ldml.fit(Y, T, X=X, W=W)
health_est_ldml.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
hiv_prevalence_pavg_lag1,0.032,0.006,5.319,0.0,0.022,0.042
fertility_pavg_lag1,-0.126,0.013,-9.386,0.0,-0.148,-0.104
gdp_pc_ppp_lag1,0.0,0.0,2.177,0.029,0.0,0.0
population_lag1,0.0,0.0,0.262,0.793,-0.0,0.0
conflict,0.818,0.0,5528.393,0.0,0.818,0.818
physicians_rate,-0.123,0.0,-297.574,0.0,-0.124,-0.122
health_share_gov_exp,-0.099,0.019,-5.187,0.0,-0.13,-0.067
log_mean_pc_last_5,0.217,0.007,32.99,0.0,0.206,0.227
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.083,0.002,35.327,0.0,0.079,0.087


In [79]:
2 ** (0.083) - 1

## WASH replications

In [ ]:
# Access = a0 + a1 * Aid + a2 * Aid^2 + beta * controls + country FE + error term

* Access (country, year) = access_water or access_sanitation  (each of them is separately used as the dependent variable, for Table 4 and Table 5, while the other is included as a lagged term in the controls). They also split urban and rural, but I think we can ignore this for now
* Aid (country, year) = Aid targeted to the water and sanitation sector as a percentage of GDP. So Aid = 100 * commit_wash / (gdp_pc * population)
* Controls (country, year): 
   - adult_literacy; log(gdp_pc), lagged(access_water or access_sanitation) and 3 others that are not exactly in the dataset but have reasonably close proxies:
   - Government spending on health (% of GDP)  is not in the dataset, but a reasonably close one is health_share_gov_exp = Government health expenditure (% of general government expenditure)
   - Age dependency ratio is not in the dataset, but a reasonably close one is young_population
   - Government stability from ICRG is not in the dataset, but reasonably close ones are conflict and freedom_house
* Other details:
   - Period = 1990-2010
   - Sample restricted to SSA countries only

In [86]:
wash_df = health_df.copy()

In [87]:
wash_df = experiment.add_project_and_aid_cols(wash_df, "wash")

wash_df['wash_aid'] = wash_df['mean_pc_last_5'] * 100 / (wash_df['gdp_pc_ppp'])
wash_df['wash_aid_sq'] = wash_df['wash_aid'] ** 2

Generating mean per capita commitments over prior years
Marking whether a satisfactory project concluded in that year
Taking maximum of weighted rating of concluded projects in prior period


In [88]:
wash_df[wash_df.wash_aid.notna()][['year', 'country', 'commit_wash', 'population', 'mean_pc_last_5', 'gdp_pc_ppp', 'wash_aid', 'wash_aid_sq']].tail()

,year,country,commit_wash,population,mean_pc_last_5,gdp_pc_ppp,wash_aid,wash_aid_sq
8674,2007,ZWE,3937525.0,12255922.0,7.661863,2133.8081,0.359070,0.128931
8843,2008,ZWE,80993723.0,12379549.0,8.221996,1739.2428,0.472734,0.223478
9012,2009,ZWE,23370839.0,12526968.0,8.442636,1925.3644,0.438495,0.192278
9181,2010,ZWE,17334374.0,12697723.0,14.347361,2273.2002,0.631153,0.398354
9350,2011,ZWE,26854875.0,12894316.0,14.023592,2556.2783,0.548594,0.300955


In [89]:
wash_df['log_gdp_pc'] = experiment.safe_log(wash_df, 'gdp_pc_ppp')
wash_df = experiment.take_avg_and_lag(wash_df, "health_share_gov_exp")
wash_df = experiment.take_avg_and_lag(wash_df, "adult_literacy")

In [90]:
wash_df['access_water_pavg'] = explore_util.rolling_country_agg(wash_df, "access_water", 5, "mean")
wash_df['access_san_pavg'] = explore_util.rolling_country_agg(wash_df, "access_sanitation", 5, "mean")

In [91]:
wash_df = explore_util.lag_variable_simple(wash_df, 'access_water_pavg', 1)
wash_df = explore_util.lag_variable_simple(wash_df, "access_san_pavg", 1)
wash_df = explore_util.lag_variable_simple(wash_df, 'access_water_pavg', 5)
wash_df = explore_util.lag_variable_simple(wash_df, "access_san_pavg", 5)

In [92]:
wash_controls_constant = ['adult_literacy_pavg', 'log_gdp_pc', 'health_share_gov_exp_pavg_lag1',
                'young_population', 'conflict', 'freedom_house']

wash_access_cols = ['access_water_pavg', 'access_san_pavg', 'access_water_pavg_lag5', 'access_san_pavg_lag5']
wash_proj_cols = ['wash_satisfactory_proj', 'wash_max_proj_5yr']

In [97]:
wash_df['log_wash_aid_sq'] = experiment.safe_log(wash_df, 'wash_aid_sq')

In [98]:
remove_for_water = ["access_san_pavg", "access_water_pavg_lag5"]
# remove_for_water = []

water_est = experiment.evaluate_treatment(wash_df[wash_df.gdp_pc_ppp < 10000], "access_water_pavg", "wash_aid",
                                          wash_controls_constant + wash_access_cols + wash_proj_cols, # + ['log_wash_aid_sq'],
                                          remove_feature_cols=remove_for_water,
                                          log_target=True, log_treatment=True, add_constant=True, 
                                          add_country_feffects=True)

experiment.extract_treatment_results('Access to Water Estimate', water_est, 'access_water_pavg', 'wash_aid', wash_controls_constant, None)

{'Label': 'Access to Water Estimate',
 'Regression P': 1.7772149560775201e-193,
 'Treatment significance': 2.651819402655426e-07,
 'Treatment coefficient': 0.06708558763132336,
 'Sig feature coefficient': {'young_population': 0.0177},
 'Sig feature p-values': {'young_population': 0.0115},
 'Number significant FE': 47,
 'Mean coefficient on FE': 2.8390223875207625,
 'Keyword args': None}

In [99]:
# effect of doubling:
print("Doubling effect: ", 2 ** (water_est.params['wash_aid']) - 1)
print(water_est.summary())

Doubling effect:  0.0475982738409817
                            OLS Regression Results                            
Dep. Variable:      access_water_pavg   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     27.63
Date:                Tue, 24 Aug 2021   Prob (F-statistic):          1.78e-193
Time:                        15:18:08   Log-Likelihood:                 500.60
No. Observations:                 857   AIC:                            -799.2
Df Residuals:                     756   BIC:                            -319.1
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [116]:
remove_for_san = ["access_water_pavg", "access_san_pavg_lag5"]
# remove_for_water = []

san_est = experiment.evaluate_treatment(wash_df[wash_df.gdp_pc_ppp < 10000], "access_san_pavg", "wash_aid",
                                          wash_controls_constant + wash_access_cols + ['w_avg_rating'], # + wash_proj_cols + ['log_wash_aid_sq'],
                                          remove_feature_cols=remove_for_san,
                                          log_target=True, log_treatment=True, add_constant=True, 
                                          add_country_feffects=True)

print(san_est.summary())

                            OLS Regression Results                            
Dep. Variable:        access_san_pavg   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     177.2
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        15:27:50   Log-Likelihood:                 449.60
No. Observations:                 859   AIC:                            -697.2
Df Residuals:                     758   BIC:                            -216.9
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [102]:
# effect of doubling here
print("Effect of doubling: ", 2 ** (san_est.params["wash_aid"]) - 1)

Effect of doubling:  0.03791408491986359


In [103]:
write_wash_results = False

if write_wash_results:
    with open("../results/intial_run/water_initial_fe_linear.txt", "w") as file:
        file.write(water_est.summary().as_text())

    with open("../results/intial_run/sanitation_initial_fe_linear.txt", "w") as file:
        file.write(san_est.summary().as_text())

#     gsearch_results.to_csv('../data/results/education_model_crawl.csv', float_format='%.4f')

In [ ]:
# future: do a pair-wise comparison, e.g., using BERT and others

### EconML

In [104]:
feature_cols = ['adult_literacy_pavg', 'log_gdp_pc', 'health_share_gov_exp_pavg_lag1',  
             'young_population', 'conflict', 'freedom_house', 'access_water_pavg_lag5']

In [119]:
wash_df['log_san_access'] = experiment.safe_log(wash_df, 'access_san_pavg')
wash_df['log_wash_aid'] = experiment.safe_log(wash_df, 'wash_aid')
wash_df['log_avg_rating'] = experiment.safe_log(wash_df, 'w_avg_rating')

In [123]:
rating_col = 'log_avg_rating'
magnitude_col = 'log_wash_aid'
target_col = 'log_san_access'

# feature_cols = [col for col in data_cols if col != treatment_col and col != target_col]
Y, T, X, W = experiment.assemble_econml_tuples(wash_df, target_col=target_col, treatment_col=magnitude_col, 
                                               feature_cols=feature_cols + [rating_col])

In [121]:
wash_est = LinearDML(model_y='auto', model_t='auto')
wash_est.fit(Y, T, X=X, W=W)
print(wash_est.summary())

                                Coefficient Results                                 
                               point_estimate stderr zstat  pvalue ci_lower ci_upper
------------------------------------------------------------------------------------
adult_literacy_pavg                    -0.001  0.001 -1.221  0.222   -0.003      0.0
log_gdp_pc                              0.015  0.007  2.038  0.042    0.003    0.026
health_share_gov_exp_pavg_lag1         -0.004  0.001 -2.902  0.004   -0.007   -0.002
young_population                       -0.004  0.001 -4.367    0.0   -0.006   -0.003
conflict                                0.068  0.033  2.069  0.038    0.014    0.122
freedom_house                          -0.005  0.005 -1.029  0.303   -0.014    0.003
access_water_pavg_lag5                 -0.003  0.001 -2.896  0.004   -0.005   -0.001
log_avg_rating                          0.029  0.011  2.739  0.006    0.012    0.047
                       CATE Intercept Results                    

In [122]:
print(wash_est.summary())

                                Coefficient Results                                 
                               point_estimate stderr zstat  pvalue ci_lower ci_upper
------------------------------------------------------------------------------------
adult_literacy_pavg                    -0.001  0.001 -1.221  0.222   -0.003      0.0
log_gdp_pc                              0.015  0.007  2.038  0.042    0.003    0.026
health_share_gov_exp_pavg_lag1         -0.004  0.001 -2.902  0.004   -0.007   -0.002
young_population                       -0.004  0.001 -4.367    0.0   -0.006   -0.003
conflict                                0.068  0.033  2.069  0.038    0.014    0.122
freedom_house                          -0.005  0.005 -1.029  0.303   -0.014    0.003
access_water_pavg_lag5                 -0.003  0.001 -2.896  0.004   -0.005   -0.001
log_avg_rating                          0.029  0.011  2.739  0.006    0.012    0.047
                       CATE Intercept Results                    

In [125]:
Y, T, X, W = experiment.assemble_econml_tuples(wash_df, target_col=target_col, treatment_col=rating_col, 
                                               feature_cols=feature_cols + [magnitude_col])

est = LinearDML(model_y='auto', model_t='auto')
est.fit(Y, T, X=X, W=W)
print(est.summary())

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7268280919428207, tolerance: 0.017648538606224697
Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3058507731746687, tolerance: 0.016652731068600955
Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.19346313939425386, tolerance: 0.01609700615149715
Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9520364112118074, tolerance: 0.01609700615149715
Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04679579828039948, tolerance: 0.018883346503693707
Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.02667053990690249, tolerance: 0.018883346503693707


                                Coefficient Results                                 
                               point_estimate stderr zstat  pvalue ci_lower ci_upper
------------------------------------------------------------------------------------
adult_literacy_pavg                    -0.005  0.001 -3.754    0.0   -0.007   -0.003
log_gdp_pc                              0.049   0.02  2.529  0.011    0.017    0.082
health_share_gov_exp_pavg_lag1          0.014  0.005  2.807  0.005    0.006    0.021
young_population                       -0.011  0.003 -3.722    0.0   -0.016   -0.006
conflict                                0.038  0.076  0.497  0.619   -0.088    0.164
freedom_house                           0.037  0.012  3.132  0.002    0.018    0.057
access_water_pavg_lag5                 -0.002  0.001 -1.339  0.181   -0.004      0.0
log_wash_aid                            0.035  0.011  3.099  0.002    0.016    0.053
                       CATE Intercept Results                    

In [126]:
wash_est = LinearDML(model_y='auto', model_t='auto')
wash_est.fit(Y, T, X=X, W=W)
print(wash_est.summary())

                                Coefficient Results                                 
                               point_estimate stderr zstat  pvalue ci_lower ci_upper
------------------------------------------------------------------------------------
adult_literacy_pavg                    -0.005  0.001 -3.658    0.0   -0.007   -0.003
log_gdp_pc                              0.047   0.02  2.431  0.015    0.015     0.08
health_share_gov_exp_pavg_lag1          0.013  0.005  2.778  0.005    0.005    0.021
young_population                       -0.011  0.003 -3.809    0.0   -0.016   -0.006
conflict                                0.018  0.075  0.237  0.812   -0.106    0.141
freedom_house                           0.037  0.012  3.181  0.001    0.018    0.057
access_water_pavg_lag5                 -0.002  0.001 -1.344  0.179   -0.004      0.0
log_wash_aid                            0.035  0.011  3.212  0.001    0.017    0.053
                       CATE Intercept Results                    